In [1]:
%cd /scratch/bruingjde/SNAM2021-code/

import os
import typing

import joblib
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
import sklearn.linear_model
import sklearn.model_selection
import sklearn.metrics
import sklearn.pipeline
import sklearn.preprocessing
from tqdm.auto import tqdm

import tlp

/scratch/bruingjde/SNAM2021-code


In [7]:
# Check
for entry in os.scandir('data'):
  if not os.path.isfile(os.path.join(entry, 'features', 'AND.pkl')): print(entry)

<DirEntry '27'>


In [6]:
# Delete!!
for entry in os.scandir('data'):
  filename = os.path.join(entry, 'features', 'CC.pkl')
  if os.path.isfile(filename): os.remove(filename)

In [7]:
def CC(path: str, verbose: bool = False, feature_name='CC'):
  # Check if file exists
  feature_path = os.path.join(path, 'features')
  file = os.path.join(feature_path, f'{feature_name}.pkl')
  if os.path.isfile(file): return 
  
  os.makedirs(feature_path, exist_ok=True)

  # Read in
  edgelist_mature_file = os.path.join(path, 'edgelist_mature.pkl')
  if verbose: print(f'Read {edgelist_mature_file}')
  edgelist_mature = pd.read_pickle(edgelist_mature_file)
  
  instances_file = os.path.join(path, 'instances_sampled.npy')
  if verbose: print(f'Read {instances_file}')  
  instances_sampled = np.load(instances_file)

  # Get all unique nodes in instances
#   nodes = {node for instance in instances_sampled for node in instance}
  
  # Get graph
  G = nx.from_pandas_edgelist(edgelist_mature)
  
  # Calculate per node
  value_per_node = nx.closeness_centrality(G)
  
  value_per_instance = [(value_per_node[u], value_per_node[v]) for u, v in instances_sampled]
  
  def diff(x): return abs(x[1]-x[0]) 
  
  results = {
    tlp.Experiment(feature=feature_name, time_aware=False, nodepair_strategy=strategy_str): (
      np.array([strategy_func(instance) for instance in value_per_instance])) 
    for strategy_str, strategy_func in {'sum': sum, 'diff': diff, 'max': max, 'min': min}.items()
  }
  
  joblib.dump(results, os.path.join(feature_path, f'{feature_name}.pkl'))

In [8]:
CC('data/13', verbose=True)

Read data/13/edgelist_mature.pkl
Read data/13/instances_sampled.npy


In [ ]:
entries = sorted(os.scandir('data'), key=lambda x: x.name)
tlp.ProgressParallel(n_jobs=len(entries), total=len(entries))(
  joblib.delayed(CC)(entry.path) for entry in entries
)

  0%|          | 0/30 [00:00<?, ?it/s]